In [47]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers
from keras.callbacks import CSVLogger
#from livelossplot import PlotLossesKeras
import os
import numpy as np
#from imgaug import augmenters as iaa
#import cv2
from keras.layers.normalization import BatchNormalization
import seaborn as sns
import pandas as pd 
from keras import initializers
from keras import optimizers
import keras.backend as K
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from keras.optimizers import SGD

In [50]:
##Change n_splits = 5 for 5 fold CV
##Sklearn StratifiedKFold doesn't work for one hot encoding hence binarized it
## so dense layer = 1 and activation = sigmoid here
## If you don't want to do it, just use model.fit(val_split = 0.2) and run 5 different models and take their test score average
## for any case don't split dataset into training/test/val you only need training set

In [48]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")
y_train = y_train[:,1:2]

In [49]:
kfold = StratifiedKFold(n_splits=2, shuffle=True)
cvscores = []
IMAGE_SIZE = 200
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE, IMAGE_SIZE
EPOCHS = 20
BATCH_SIZE = 32
input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)

for train, test in kfold.split(x_train, y_train):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Dropout(0.5))

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))



    
    opt = SGD(lr=0.01)
    model.compile(loss = "binary_crossentropy", optimizer = opt,  metrics=['accuracy'])

    model.fit(x=x_train[train], y=y_train[train], batch_size = BATCH_SIZE, epochs=EPOCHS, verbose=1)
    scores = model.evaluate(x_train[test], y_train[test], verbose=0)
    cvscores.append(scores[1])
print('Total CV score is {}'.format(np.mean(cvscores)))
print('Total CV score std is {}'.format(np.std(cvscores)))

Epoch 1/20
11023/11023 [==============================] - 93s 8ms/step - loss: 0.8105 - acc: 0.6902
Epoch 2/20
11023/11023 [==============================] - 91s 8ms/step - loss: 0.2246 - acc: 0.9198
Epoch 3/20
11023/11023 [==============================] - 91s 8ms/step - loss: 0.1709 - acc: 0.9420
Epoch 4/20
11023/11023 [==============================] - 92s 8ms/step - loss: 0.1522 - acc: 0.9495
Epoch 5/20
11023/11023 [==============================] - 92s 8ms/step - loss: 0.1415 - acc: 0.9522
Epoch 6/20
11023/11023 [==============================] - 92s 8ms/step - loss: 0.1241 - acc: 0.9564
Epoch 7/20
11023/11023 [==============================] - 91s 8ms/step - loss: 0.1193 - acc: 0.9595
Epoch 8/20
11023/11023 [==============================] - 91s 8ms/step - loss: 0.1064 - acc: 0.9643
Epoch 9/20
11023/11023 [==============================] - 91s 8ms/step - loss: 0.1040 - acc: 0.9638
Epoch 10/20
11023/11023 [==============================] - 91s 8ms/step - loss: 0.0866 - acc: 0.9690